In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
file_review3="C:/Users/Lu/PycharmProjects/Group_Project/orig_review_with_labeling_608598rows.txt"
review3=pd.read_csv(file_review3,sep="\t",header=None)
review3.columns=['user_id', 'prod_id', 'date', 'review', 'rating', 'label']
print(review3.head())
print("length of data:",len(review3))
print("check the null data:",review3.isnull().values.any())

   user_id  prod_id        date  \
0     5044        0  2014-11-16   
1     5045        0  2014-09-08   
2     5046        0  2013-10-06   
3     5047        0  2014-11-30   
4     5048        0  2014-08-28   

                                              review  rating  label  
0  Drinks were bad, the hot chocolate was watered...     1.0     -1  
1  This was the worst experience I've ever had a ...     1.0     -1  
2  This is located on the site of the old Spruce ...     3.0     -1  
3  I enjoyed coffee and breakfast twice at Toast ...     5.0     -1  
4  I love Toast! The food choices are fantastic -...     5.0     -1  
length of data: 608598
check the null data: False


In [81]:
data=review3[['user_id','date','review']]
print(data.head())

   user_id        date                                             review
0     5044  2014-11-16  Drinks were bad, the hot chocolate was watered...
1     5045  2014-09-08  This was the worst experience I've ever had a ...
2     5046  2013-10-06  This is located on the site of the old Spruce ...
3     5047  2014-11-30  I enjoyed coffee and breakfast twice at Toast ...
4     5048  2014-08-28  I love Toast! The food choices are fantastic -...


In [82]:
data['date'] = pd.to_datetime(data['date'])

C:\Users\Lu\AppData\Local\Temp\ipykernel_22964\736794659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])


In [83]:
print(data.head())

   user_id       date                                             review
0     5044 2014-11-16  Drinks were bad, the hot chocolate was watered...
1     5045 2014-09-08  This was the worst experience I've ever had a ...
2     5046 2013-10-06  This is located on the site of the old Spruce ...
3     5047 2014-11-30  I enjoyed coffee and breakfast twice at Toast ...
4     5048 2014-08-28  I love Toast! The food choices are fantastic -...


In [84]:
print(type(data.iloc[0,1]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [85]:
data=data.sort_values('user_id')

In [86]:
print(data.iloc[5:20,:])

        user_id       date                                             review
2          5046 2013-10-06  This is located on the site of the old Spruce ...
3          5047 2014-11-30  I enjoyed coffee and breakfast twice at Toast ...
4          5048 2014-08-28  I love Toast! The food choices are fantastic -...
5          5049 2013-07-16  The egg on an English muffin (their take on eg...
6          5050 2013-03-03  wonderful relaxed vibe and fantastic homemade ...
7          5051 2014-12-05  Extremely slow kitchen. I went with an hour to...
127448     5052 2014-11-26  I had the turducken and Alex had the angus bur...
353887     5052 2014-12-17  Absolutely adored this place. You see, I'm a p...
8          5052 2014-11-26  I really wanted to love toast. It's quaint and...
427040     5053 2014-09-20  I have to say, being healthy takes a huge toll...
137661     5053 2014-12-13  The trip down to Sansom is so worth it! The su...
132806     5053 2014-08-20  Awesome soba salad here! My friend o

In [87]:
df=data.copy()

In [88]:
df2=df.groupby([df.user_id, df.date])['review'].count().max(level=0)

C:\Users\Lu\AppData\Local\Temp\ipykernel_22964\1620777842.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  df2=df.groupby([df.user_id, df.date])['review'].count().max(level=0)


### Get maximum number of reviews posted in one day by a reviewer

In [104]:
print(df2.head(10)) # this is pandas series, not pandas dataframe

user_id
5044    1
5045    1
5046    1
5047    1
5048    1
5049    1
5050    1
5051    1
5052    2
5053    2
Name: review, dtype: int64


In [105]:
df3=df2.to_frame()
print(df3.head())

         review
user_id        
5044          1
5045          1
5046          1
5047          1
5048          1


In [106]:
df3.reset_index(inplace=True)
print(df3.columns)

Index(['user_id', 'review'], dtype='object')


In [107]:
df3=df3.rename(columns={'user_id':'user_id', 'review':'max_reviews_one_day'})

In [108]:
print(df3.head(10))

   user_id  max_reviews_one_day
0     5044                    1
1     5045                    1
2     5046                    1
3     5047                    1
4     5048                    1
5     5049                    1
6     5050                    1
7     5051                    1
8     5052                    2
9     5053                    2


In [110]:
print(review3.head())

   user_id  prod_id        date  \
0     5044        0  2014-11-16   
1     5045        0  2014-09-08   
2     5046        0  2013-10-06   
3     5047        0  2014-11-30   
4     5048        0  2014-08-28   

                                              review  rating  label  
0  Drinks were bad, the hot chocolate was watered...     1.0     -1  
1  This was the worst experience I've ever had a ...     1.0     -1  
2  This is located on the site of the old Spruce ...     3.0     -1  
3  I enjoyed coffee and breakfast twice at Toast ...     5.0     -1  
4  I love Toast! The food choices are fantastic -...     5.0     -1  


### Left join review3 with max_reviews_one_day

In [111]:
final_data=review3.merge(df3, on='user_id',how='left')

In [113]:
print(final_data.head(10))

   user_id  prod_id        date  \
0     5044        0  2014-11-16   
1     5045        0  2014-09-08   
2     5046        0  2013-10-06   
3     5047        0  2014-11-30   
4     5048        0  2014-08-28   
5     5049        0  2013-07-16   
6     5050        0  2013-03-03   
7     5051        0  2014-12-05   
8     5052        0  2014-11-26   
9     5053        0  2014-11-17   

                                              review  rating  label  \
0  Drinks were bad, the hot chocolate was watered...     1.0     -1   
1  This was the worst experience I've ever had a ...     1.0     -1   
2  This is located on the site of the old Spruce ...     3.0     -1   
3  I enjoyed coffee and breakfast twice at Toast ...     5.0     -1   
4  I love Toast! The food choices are fantastic -...     5.0     -1   
5  The egg on an English muffin (their take on eg...     5.0     -1   
6  wonderful relaxed vibe and fantastic homemade ...     5.0     -1   
7  Extremely slow kitchen. I went with an hour

In [114]:
print(len(final_data))

608598


In [115]:
from numpy import savetxt
savetxt("feature_max_reviews_one_day.txt",final_data,fmt="%s",delimiter="\t",encoding="utf-8")